In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, MaxPool2D, AveragePooling2D, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# 데이터 로드
mnist = tf.keras.datasets.mnist

In [3]:
# 데이터 전처리
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # 데이터 정규화
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [4]:
# ResNet 블록 클래스 정의
class ResNetBlock(Layer):
    def __init__(self, num_filters, bottleneck_size):
        super(ResNetBlock, self).__init__()
        self.conv1 = Conv2D(bottleneck_size, (1, 1), activation='relu')
        self.conv2 = Conv2D(bottleneck_size, (3, 3), padding='same', activation='relu')
        self.conv3 = Conv2D(num_filters, (1, 1), activation='relu')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        return tf.nn.relu(x + inputs)  # Residual connection

In [8]:
class ResNetBlock(Layer):
    def __init__(self, num_filters, bottleneck_size):
        super(ResNetBlock, self).__init__()
        self.conv1 = Conv2D(bottleneck_size, (1, 1), activation='relu')
        self.conv2 = Conv2D(bottleneck_size, (3, 3), padding='same', activation='relu')
        self.conv3 = Conv2D(num_filters, (1, 1), activation='relu')
        self.adjust_dims = Conv2D(num_filters, (1, 1), activation='relu')  # 차원 조정을 위한 추가 컨볼루션

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        inputs_adjusted = self.adjust_dims(inputs)  # 입력 차원을 출력과 일치시킴
        return tf.nn.relu(x + inputs_adjusted)  # 잔차 연결

In [9]:
# 모델 인스턴스 생성 및 컴파일
model = ResNetModel()
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
# 훈련 및 평가
model.fit(x_train, y_train, epochs=10, batch_size=100)
model.evaluate(x_test, y_test)

Epoch 1/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 285s 465ms/step - accuracy: 0.7643 - loss: 0.7324
Epoch 2/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 329s 548ms/step - accuracy: 0.9762 - loss: 0.0768
Epoch 3/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 405s 674ms/step - accuracy: 0.9835 - loss: 0.0531
Epoch 4/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 316s 526ms/step - accuracy: 0.9870 - loss: 0.0419
Epoch 5/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 309s 514ms/step - accuracy: 0.9899 - loss: 0.0339
Epoch 6/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 313s 521ms/step - accuracy: 0.9911 - loss: 0.0284
Epoch 7/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1258s 2s/step - accuracy: 0.9923 - loss: 0.0253
Epoch 8/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 606s 1s/step - accuracy: 0.9929 - loss: 0.0222
Epoch 9/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 619s 1s/step - accuracy: 0.9940 - loss: 0.0188
Epoch 10/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 608s 1s/step - accuracy: 0.9940 - loss: 0.0178
313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 119ms/step - accuracy: 0.9895 - loss: 0.0306


[0.023996150121092796, 0.9918000102043152]

- Loss: 0.023996150121092796 <br>
- Accuracy: 0.9918000102043152